In [7]:
%pip install 'unified_planning[grpc]'

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.3/713.3 kB 38.7 MB/s eta 0:00:00
  Created wheel for ConfigSpace: filename=configspace-1.2.1-py3-none-any.whl size=115990 sha256=36ede31f285d0b06d0b99203200ef6c32e2e2d88cd3e972691629d37d34b29c5
  Stored in directory: /Users/jesusvillalbafernandez/Library/Caches/pip/wheels/11/0f/36/d5027c3eeb038827889830f7efbe6a1bad8956b3eb44ab2f44
Successfully built ConfigSpace
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install up-enhsp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 MB 47.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from unified_planning.shortcuts import *
import unified_planning as up

# Silenciar créditos del planner
up.shortcuts.get_environment().credits_stream = None

def crear_problema_lightsout(tamaño: int) -> Problem:
    Celda = UserType('Celda')
    problema = Problem('LightsOut')

    estado = Fluent('estado', BoolType(), c=Celda)
    vecino = Fluent('vecino', BoolType(), c1=Celda, c2=Celda)

    problema.add_fluent(estado, default_initial_value=False)
    problema.add_fluent(vecino, default_initial_value=False)

    pulsar = InstantaneousAction('pulsar', c=Celda)
    c = pulsar.parameter('c')
    pulsar.add_effect(estado(c), Not(estado(c)))
    problema.add_action(pulsar)

    celdas = [Object(f'[{i},{j}]', Celda) for i in range(tamaño) for j in range(tamaño)]
    problema.add_objects(celdas)

    for i in range(tamaño):
        for j in range(tamaño):
            c_actual = next(obj for obj in celdas if obj.name == f'[{i},{j}]')
            vecinos = []
            if i > 0: vecinos.append((i-1, j))
            if i < tamaño - 1: vecinos.append((i+1, j))
            if j > 0: vecinos.append((i, j-1))
            if j < tamaño - 1: vecinos.append((i, j+1))
            for vi, vj in vecinos:
                c_vecino = next(obj for obj in celdas if obj.name == f'[{vi},{vj}]')
                problema.set_initial_value(vecino(c_actual, c_vecino), True)

    for c1 in celdas:
        for c2 in celdas:
            cond = And(Equals(pulsar.parameter('c'), c1), vecino(c1, c2))
            pulsar.add_effect(estado(c2), Not(estado(c2)), cond)

    for c_obj in celdas:
        problema.set_initial_value(estado(c_obj), False)
        problema.add_goal(estado(c_obj))

    return problema

def resolver_lightsout(tamaño: int):
    problema = crear_problema_lightsout(tamaño)

    print(f"\nResolviendo LightsOut de tamaño {tamaño}x{tamaño}...\n")

    with OneshotPlanner(name='enhsp') as planner:  
        resultado = planner.solve(problema)
        if resultado.status == up.engines.PlanGenerationResultStatus.SOLVED_SATISFICING:
            print(f"Plan encontrado en {len(resultado.plan.actions)} pasos:\n")
            for i, accion in enumerate(resultado.plan.actions, 1):
                print(f"Paso {i}: {accion}")
        else:
            print("No se encontró plan para este problema.")

# Tamaño del tablero
resolver_lightsout(tamaño=4)


Resolviendo LightsOut de tamaño 4x4...

Plan encontrado en 8 pasos:

Paso 1: pulsar([2,2])
Paso 2: pulsar([0,1])
Paso 3: pulsar([3,0])
Paso 4: pulsar([2,3])
Paso 5: pulsar([1,3])
Paso 6: pulsar([1,2])
Paso 7: pulsar([0,1])
Paso 8: pulsar([0,0])


In [12]:
from collections import deque

def crear_vecinos(tamaño):
    vecinos = {}
    for i in range(tamaño):
        for j in range(tamaño):
            pos = f'[{i},{j}]'
            vecinos[pos] = []
            if i > 0:
                vecinos[pos].append(f'[{i-1},{j}]')
            if i < tamaño - 1:
                vecinos[pos].append(f'[{i+1},{j}]')
            if j > 0:
                vecinos[pos].append(f'[{i},{j-1}]')
            if j < tamaño - 1:
                vecinos[pos].append(f'[{i},{j+1}]')
    return vecinos

def aplicar_accion(estado_actual, celda_pulsada, vecinos):
    nuevo_estado = estado_actual.copy()
    # Invertir la celda pulsada
    nuevo_estado[celda_pulsada] = not nuevo_estado[celda_pulsada]
    # Invertir vecinos
    for v in vecinos[celda_pulsada]:
        nuevo_estado[v] = not nuevo_estado[v]
    return nuevo_estado

def es_objetivo(estado_actual):
    return all(estado_actual.values())

def similitud(estado_actual):
    return sum(estado_actual.values())

def busqueda_heuristica(tamaño, n=3):
    celdas = [f'[{i},{j}]' for i in range(tamaño) for j in range(tamaño)]
    vecinos = crear_vecinos(tamaño)
    estado_inicial = {c: False for c in celdas}

    cola = deque()
    cola.append( (estado_inicial, []) )
    visitados = set()

    while cola:
        estado_actual, plan = cola.popleft()
        estado_frozenset = frozenset(estado_actual.items())
        if estado_frozenset in visitados:
            continue
        visitados.add(estado_frozenset)

        if es_objetivo(estado_actual):
            return plan

        sucesores = []
        for c in celdas:
            nuevo_estado = aplicar_accion(estado_actual, c, vecinos)
            nuevo_plan = plan + [f'pulsar {c}']
            sucesores.append( (nuevo_estado, nuevo_plan) )

        # Si alguno es objetivo, devolvemos inmediatamente
        for est, p in sucesores:
            if es_objetivo(est):
                return p

        # Tomamos los n mejores sucesores según similitud
        sucesores.sort(key=lambda x: similitud(x[0]), reverse=True)
        cola.extend(sucesores[:n])

    return None

# Prueba con tablero 3x3 y n=3
plan = busqueda_heuristica(4, n=6)
if plan:
    print(f"Plan encontrado en {len(plan)} pasos:")
    for i, paso in enumerate(plan, 1):
        print(f"Paso {i}: {paso}")
else:
    print("No se encontró plan con esta búsqueda heurística.")

Plan encontrado en 4 pasos:
Paso 1: pulsar [0,1]
Paso 2: pulsar [1,3]
Paso 3: pulsar [2,0]
Paso 4: pulsar [3,2]
